In [1]:
!sudo apt-get install texlive-xetex texlive-fonts-recommended texlive-plain-generic
!jupyter nbconvert --to pdf /content/Data_analysis.ipynb

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  dvisvgm fonts-droid-fallback fonts-lato fonts-lmodern fonts-noto-mono
  fonts-texgyre fonts-urw-base35 libapache-pom-java libcommons-logging-java
  libcommons-parent-java libfontbox-java libfontenc1 libgs9 libgs9-common
  libidn12 libijs-0.35 libjbig2dec0 libkpathsea6 libpdfbox-java libptexenc1
  libruby3.0 libsynctex2 libteckit0 libtexlua53 libtexluajit2 libwoff1
  libzzip-0-13 lmodern poppler-data preview-latex-style rake ruby
  ruby-net-telnet ruby-rubygems ruby-webrick ruby-xmlrpc ruby3.0
  rubygems-integration t1utils teckit tex-common tex-gyre texlive-base
  texlive-binaries texlive-latex-base texlive-latex-extra
  texlive-latex-recommended texlive-pictures tipa xfonts-encodings
  xfonts-utils
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf libavalon-framework-java
  libcommons-logging-java-doc libexcalibu

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data Cleaning

In [ ]:
pip install ijson

In [ ]:
import ijson
import os
import json
from decimal import Decimal

def process_json_files(start, end):
  input_file_path = f"/content/profiles_2021-11-10_{start}_{end}.json"
  output_dir = f"/content/{start}_{end}"

  os.makedirs(output_dir, exist_ok=True)

  def custom_serializer(obj):
    if isinstance(obj, Decimal):
      return float(obj)
    raise TypeError("Type %s not serializable" % type(obj))

  with open(input_file_path, 'r') as file:
    items = ijson.items(file, 'item')
    for index, item in enumerate(items):
      output_filename = item.get('_id', f"default_{index}")
      output_path = os.path.join(output_dir, f"{output_filename}.json")

      with open(output_path, 'w') as out_file:
        json.dump(item, out_file, indent=4, default=custom_serializer)

process_json_files(1, 100)

# Data Preprocessing

In [ ]:
import pandas as pd
df=pd.read_csv('/content/drive/MyDrive/CS 571 K Night Stand/truncated.csv')
df.head(5)

,Unnamed: 0,__v,_id,conversations,conversationsMeta.averageConversationLength,conversationsMeta.averageConversationLengthInDays,conversationsMeta.longestConversation,conversationsMeta.longestConversationInDays,conversationsMeta.medianConversationLength,conversationsMeta.medianConversationLengthInDays,...,user.education,user.educationLevel,user.gender,user.genderFilter,user.instagram,user.interestedIn,user.jobs,user.schools,user.spotify,userId
0,0,0,00b74e27ad1cbb2ded8e907fcc49eaaf,"[{""match_id"":""Match 739"",""messages"":[{""to"":738...",8.560217,10.236620,133,683.557442,3,0.081134,...,Has high school and/or college education,Has high school and/or college education,M,F,False,F,[],[],False,00b74e27ad1cbb2ded8e907fcc49eaaf
1,1,0,024610702baf540af5637873cd1534e9,"[{""match_id"":""Match 464"",""messages"":[{""to"":463...",4.495690,3.973980,36,622.466528,2,0.000069,...,Has high school and/or college education,Has high school and/or college education,M,F,False,F,[],[],False,024610702baf540af5637873cd1534e9
2,2,0,0a5e3dd8489fe67485ddb7d6adb26ebd,"[{""match_id"":""Match 303"",""messages"":[{""to"":302...",4.254125,3.465479,33,295.283414,2,0.033715,...,Has no high school or college education,Has no high school or college education,M,F,False,F,[],[],False,0a5e3dd8489fe67485ddb7d6adb26ebd
3,3,0,048dd37565ad9cbc24c163ffedffbf58,"[{""match_id"":""Match 47"",""messages"":[{""to"":46,""...",7.893617,5.878985,28,130.034410,6,0.643727,...,Has no high school or college education,Has no high school or college education,M,F,False,F,[],[],False,048dd37565ad9cbc24c163ffedffbf58
4,4,0,0eb998fdde77f9c123c07eace18a5cc1,"[{""match_id"":""Match 809"",""messages"":[{""to"":808...",6.824475,1.924996,444,198.390972,3,0.091771,...,Has no high school or college education,Has no high school or college education,M,F,True,F,"[{""companyDisplayed"":false,""titleDisplayed"":tr...","[{""displayed"":true,""name"":""Humboldt-Universitä...",False,0eb998fdde77f9c123c07eace18a5cc1


In [ ]:
pip install transformers nltk

In [ ]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
# neg: Negative score, represents the proportion of text that contributes to a negative sentiment.
# neu: Neutral score, represents the proportion of text that is neutral.
# pos: Positive score, represents the proportion of text that contributes to a positive sentiment.
# compound: A compound score that computes the sum of all lexicon ratings normalized between -1 (most negative) and +1 (most positive).
import os
import json
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from transformers import pipeline

sia = SentimentIntensityAnalyzer()
emotion_classifier = pipeline("text-classification", model='bhadresh-savani/bert-base-uncased-emotion', return_all_scores=True)

input_folder_path = '/content/drive/MyDrive/CS 571 K Night Stand/Data/401_500_rest/'
output_folder_path = '/content/drive/MyDrive/CS 571 K Night Stand/Data/results/401_500/'
os.makedirs(output_folder_path, exist_ok=True)

for filename in os.listdir(input_folder_path):
    if filename.endswith(".json"):
        file_path = os.path.join(input_folder_path, filename)
        with open(file_path, 'r') as file:
            data = json.load(file)

        if 'conversations' in data:
            results = []

            for conversation in data['conversations']:
                if 'messages' in conversation:
                    conversation_result = {
                        "Match ID": conversation['match_id'],
                        "Messages": []
                    }
                    print(f"Analyzing Match ID: {conversation['match_id']}")
                    for message in conversation['messages']:
                        if 'message' in message:
                            sentiment_scores = sia.polarity_scores(message['message'])
                            emotion_prediction = emotion_classifier(message['message'])
                            message_result = {
                                "Message": message['message'],
                                "Sentiment Scores": sentiment_scores,
                                "Emotion Scores": emotion_prediction
                            }
                            conversation_result["Messages"].append(message_result)
                    results.append(conversation_result)

            output_file_path = os.path.join(output_folder_path, filename)
            with open(output_file_path, 'w') as outfile:
                json.dump(results, outfile, indent=4)

            print(f"Results for {filename} saved to {output_file_path}")
        else:
            print(f"No 'conversations' in {filename}, skipping file.")

In [ ]:
import os
import json

input_folder_path = '/content/drive/MyDrive/CS 571 K Night Stand/Data/results/401_500/'
output_file_path = '/content/drive/MyDrive/CS 571 K Night Stand/Data/results/401_500_summary_scores.json'

all_results = []

for filename in os.listdir(input_folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(input_folder_path, filename)
        with open(file_path, 'r') as file:
            data = json.load(file)

        total_sentiment_scores = {"neg": 0, "neu": 0, "pos": 0, "compound": 0}
        total_emotion_scores = {}
        message_count = 0

        for matches in data:
            if 'Messages' in matches:
                for message in matches['Messages']:
                    if 'Sentiment Scores' in message:
                        for key in message['Sentiment Scores']:
                            total_sentiment_scores[key] += message['Sentiment Scores'][key]

                    if 'Emotion Scores' in message:
                        for emotion_list in message['Emotion Scores']:
                            for emotion in emotion_list:
                                if 'label' in emotion and 'score' in emotion:
                                    if emotion['label'] not in total_emotion_scores:
                                        total_emotion_scores[emotion['label']] = 0
                                    total_emotion_scores[emotion['label']] += emotion['score']

                    message_count += 1

        if message_count > 0:
            average_sentiment_scores = {key: value / message_count for key, value in total_sentiment_scores.items()}
            total_emotion_score = sum(total_emotion_scores.values())
            normalized_emotion_scores = {key: value / total_emotion_score for key, value in total_emotion_scores.items()}

        all_results.append({
            "filename": filename,
            "average_sentiment_scores": average_sentiment_scores,
            "normalized_emotion_scores": normalized_emotion_scores
        })

with open(output_file_path, 'w') as outfile:
    json.dump(all_results, outfile, indent=4)

print(f"Results saved to {output_file_path}")


Results saved to /content/drive/MyDrive/CS 571 K Night Stand/Data/results/401_500_summary_scores.json


In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/CS 571 K Night Stand/truncated_test.csv')
df_test.head(5)

,Unnamed: 0,__v,_id,conversations,conversationsMeta.averageConversationLength,conversationsMeta.averageConversationLengthInDays,conversationsMeta.longestConversation,conversationsMeta.longestConversationInDays,conversationsMeta.medianConversationLength,conversationsMeta.medianConversationLengthInDays,...,user.education,user.educationLevel,user.gender,user.genderFilter,user.instagram,user.interestedIn,user.jobs,user.schools,user.spotify,userId
0,0,0,00b74e27ad1cbb2ded8e907fcc49eaaf,"[{""match_id"":""Match 739"",""messages"":[{""to"":738...",8.560217,10.236620,133,683.557442,3,0.081134,...,Has high school and/or college education,Has high school and/or college education,M,F,False,F,[],[],False,00b74e27ad1cbb2ded8e907fcc49eaaf
1,1,0,024610702baf540af5637873cd1534e9,"[{""match_id"":""Match 464"",""messages"":[{""to"":463...",4.495690,3.973980,36,622.466528,2,0.000069,...,Has high school and/or college education,Has high school and/or college education,M,F,False,F,[],[],False,024610702baf540af5637873cd1534e9
2,2,0,0a5e3dd8489fe67485ddb7d6adb26ebd,"[{""match_id"":""Match 303"",""messages"":[{""to"":302...",4.254125,3.465479,33,295.283414,2,0.033715,...,Has no high school or college education,Has no high school or college education,M,F,False,F,[],[],False,0a5e3dd8489fe67485ddb7d6adb26ebd
3,3,0,048dd37565ad9cbc24c163ffedffbf58,"[{""match_id"":""Match 47"",""messages"":[{""to"":46,""...",7.893617,5.878985,28,130.034410,6,0.643727,...,Has no high school or college education,Has no high school or college education,M,F,False,F,[],[],False,048dd37565ad9cbc24c163ffedffbf58
4,4,0,0eb998fdde77f9c123c07eace18a5cc1,"[{""match_id"":""Match 809"",""messages"":[{""to"":808...",6.824475,1.924996,444,198.390972,3,0.091771,...,Has no high school or college education,Has no high school or college education,M,F,True,F,"[{""companyDisplayed"":false,""titleDisplayed"":tr...","[{""displayed"":true,""name"":""Humboldt-Universitä...",False,0eb998fdde77f9c123c07eace18a5cc1


In [ ]:
import pandas as pd
import json

csv_file_path = '/content/drive/MyDrive/CS 571 K Night Stand/truncated_test.csv'
df = pd.read_csv(csv_file_path)

json_file_path = '/content/drive/MyDrive/CS 571 K Night Stand/full_summary.json'
with open(json_file_path, 'r') as file:
    results = json.load(file)

data = []
for item in results:
    filename = item['filename'].replace('.json', '')
    record = {
        '_id': filename,
        'neg': item['average_sentiment_scores'].get('neg', 0) if 'average_sentiment_scores' in item else 0,
        'neu': item['average_sentiment_scores'].get('neu', 0) if 'average_sentiment_scores' in item else 0,
        'pos': item['average_sentiment_scores'].get('pos', 0) if 'average_sentiment_scores' in item else 0,
        'compound': item['average_sentiment_scores'].get('compound', 0) if 'average_sentiment_scores' in item else 0,
        'sadness': item['normalized_emotion_scores'].get('sadness', 0) if 'normalized_emotion_scores' in item else 0,
        'joy': item['normalized_emotion_scores'].get('joy', 0) if 'normalized_emotion_scores' in item else 0,
        'love': item['normalized_emotion_scores'].get('love', 0) if 'normalized_emotion_scores' in item else 0,
        'anger': item['normalized_emotion_scores'].get('anger', 0) if 'normalized_emotion_scores' in item else 0,
        'fear': item['normalized_emotion_scores'].get('fear', 0) if 'normalized_emotion_scores' in item else 0,
        'surprise': item['normalized_emotion_scores'].get('surprise', 0) if 'normalized_emotion_scores' in item else 0
    }
    data.append(record)

new_df = pd.DataFrame(data)

merged_df = df.merge(new_df, how='left', on='_id')

merged_df.to_csv(csv_file_path, index=False)

print("CSV file has been updated with sentiment and emotion scores.")

CSV file has been updated with sentiment and emotion scores.


In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/CS 571 K Night Stand/truncated_test.csv')
df_test.head(5)

,Unnamed: 0,__v,_id,conversations,conversationsMeta.averageConversationLength,conversationsMeta.averageConversationLengthInDays,conversationsMeta.longestConversation,conversationsMeta.longestConversationInDays,conversationsMeta.medianConversationLength,conversationsMeta.medianConversationLengthInDays,...,neg,neu,pos,compound,sadness,joy,love,anger,fear,surprise
0,0,0,00b74e27ad1cbb2ded8e907fcc49eaaf,"[{""match_id"":""Match 739"",""messages"":[{""to"":738...",8.560217,10.236620,133,683.557442,3,0.081134,...,0.037953,0.862745,0.096905,0.115272,0.048039,0.651042,0.028687,0.212246,0.048507,0.011479
1,1,0,024610702baf540af5637873cd1534e9,"[{""match_id"":""Match 464"",""messages"":[{""to"":463...",4.495690,3.973980,36,622.466528,2,0.000069,...,0.037953,0.862745,0.096905,0.115272,0.048039,0.651042,0.028687,0.212246,0.048507,0.011479
2,2,0,0a5e3dd8489fe67485ddb7d6adb26ebd,"[{""match_id"":""Match 303"",""messages"":[{""to"":302...",4.254125,3.465479,33,295.283414,2,0.033715,...,0.051830,0.786202,0.159631,0.146517,0.067231,0.521251,0.048713,0.294467,0.058183,0.010155
3,3,0,048dd37565ad9cbc24c163ffedffbf58,"[{""match_id"":""Match 47"",""messages"":[{""to"":46,""...",7.893617,5.878985,28,130.034410,6,0.643727,...,0.053480,0.782474,0.161342,0.270795,0.078426,0.626561,0.024809,0.185712,0.070238,0.014254
4,4,0,0eb998fdde77f9c123c07eace18a5cc1,"[{""match_id"":""Match 809"",""messages"":[{""to"":808...",6.824475,1.924996,444,198.390972,3,0.091771,...,0.051830,0.786202,0.159631,0.146517,0.067231,0.521251,0.048713,0.294467,0.058183,0.010155


In [ ]:
column_names = ['neg', 'neu', 'pos', 'compound', 'sadness', 'joy', 'love', 'anger', 'fear', 'surprise']

for column_name in column_names:
  non_zero_count = df_test[column_name].notna().sum()
  print(f"Number of non-zero entries in column '{column_name}': {non_zero_count}")

Number of non-zero entries in column 'neg': 597
Number of non-zero entries in column 'neu': 597
Number of non-zero entries in column 'pos': 597
Number of non-zero entries in column 'compound': 597
Number of non-zero entries in column 'sadness': 597
Number of non-zero entries in column 'joy': 597
Number of non-zero entries in column 'love': 597
Number of non-zero entries in column 'anger': 597
Number of non-zero entries in column 'fear': 597
Number of non-zero entries in column 'surprise': 597


In [ ]:
df_test.head(20)

,Unnamed: 0,__v,_id,conversations,conversationsMeta.averageConversationLength,conversationsMeta.averageConversationLengthInDays,conversationsMeta.longestConversation,conversationsMeta.longestConversationInDays,conversationsMeta.medianConversationLength,conversationsMeta.medianConversationLengthInDays,...,neg,neu,pos,compound,sadness,joy,love,anger,fear,surprise
0,0,0,00b74e27ad1cbb2ded8e907fcc49eaaf,"[{""match_id"":""Match 739"",""messages"":[{""to"":738...",8.560217,10.236620,133,683.557442,3,0.081134,...,0.037953,0.862745,0.096905,0.115272,0.048039,0.651042,0.028687,0.212246,0.048507,0.011479
1,1,0,024610702baf540af5637873cd1534e9,"[{""match_id"":""Match 464"",""messages"":[{""to"":463...",4.495690,3.973980,36,622.466528,2,0.000069,...,0.037953,0.862745,0.096905,0.115272,0.048039,0.651042,0.028687,0.212246,0.048507,0.011479
2,2,0,0a5e3dd8489fe67485ddb7d6adb26ebd,"[{""match_id"":""Match 303"",""messages"":[{""to"":302...",4.254125,3.465479,33,295.283414,2,0.033715,...,0.051830,0.786202,0.159631,0.146517,0.067231,0.521251,0.048713,0.294467,0.058183,0.010155
3,3,0,048dd37565ad9cbc24c163ffedffbf58,"[{""match_id"":""Match 47"",""messages"":[{""to"":46,""...",7.893617,5.878985,28,130.034410,6,0.643727,...,0.053480,0.782474,0.161342,0.270795,0.078426,0.626561,0.024809,0.185712,0.070238,0.014254
4,4,0,0eb998fdde77f9c123c07eace18a5cc1,"[{""match_id"":""Match 809"",""messages"":[{""to"":808...",6.824475,1.924996,444,198.390972,3,0.091771,...,0.051830,0.786202,0.159631,0.146517,0.067231,0.521251,0.048713,0.294467,0.058183,0.010155
5,5,0,10a8c197447a3991d64136b315d6e87e,"[{""match_id"":""Match 122"",""messages"":[{""to"":121...",22.688525,2.703524,808,98.055521,5,0.246412,...,0.029742,0.920891,0.024800,-0.005984,0.027696,0.209861,0.026955,0.646253,0.083311,0.005923
6,6,0,1a6bc90a124bed79741b441929c4d365,"[{""match_id"":""Match 258"",""messages"":[{""to"":257...",10.127907,15.819797,133,533.196586,3,0.099387,...,0.037953,0.862745,0.096905,0.115272,0.048039,0.651042,0.028687,0.212246,0.048507,0.011479
7,7,0,1c2f3d5f9d2ca312136fdf6e018a177d,"[{""match_id"":""Match 79"",""messages"":[{""to"":78,""...",3.658228,1.354183,23,17.015880,2,0.007523,...,0.048631,0.700675,0.217309,0.164742,0.057754,0.452684,0.034316,0.364488,0.074959,0.015799
8,8,0,1e74b4cb4b126a8f34b2d9b1fda0a92f,"[{""match_id"":""Match 663"",""messages"":[]},{""matc...",8.119155,4.407375,123,211.174363,4,0.036007,...,0.032515,0.838247,0.118465,0.106037,0.060642,0.387340,0.018125,0.435464,0.091311,0.007118
9,9,0,2057ea510896a025db5790675c90b7d6,"[{""match_id"":""Match 1580"",""messages"":[{""to"":15...",4.789241,13.644048,80,1451.564838,1,0.000000,...,0.021053,0.729909,0.244675,0.219119,0.035773,0.605254,0.026807,0.265498,0.059253,0.007415
